In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os

def kregression(data):
    #——编写k的求解程序----#
    #---solve k in this function----#
    k = 1
    return k

In [ ]:
if __name__ == '__main__':
    #-------------读取训练集,训练集地址已经设定好，不用修改------------------#
    #--------Read the training set, the address of the training set has been set and does not need to be modified------------#
    train_path = "/bohr/train-0ahg/v1/data_train.csv"
    data_train = pd.read_csv(train_path)

    #-------------读取测试集，“DATA_PATH”是测试集加密后的环境变量，按照如下方式可以在提交后，系统评分时访问测试集，但是选手无法直接下载---------#
    # Read the test set, “DATA_PATH” is the environment variable encrypted for the test set. It can be accessed during system scoring after submission as follows, but the participants cannot directly download it.#
    if os.environ.get('DATA_PATH'):
        DATA_PATH = os.environ.get("DATA_PATH") + "/"
    else:
        print("Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象")  #Baseline运行时，因为无法读取测试集，所以会有此条报错，属于正常现象
        print("When baseline is running, this error message will appear because the test set cannot be read, which is a normal phenomenon.")
    test_path = DATA_PATH + "data_test.csv"
    data_test = pd.read_csv(test_path)
    k_train = kregression(data_train)
    k_test = kregression(data_test)
    k = [k_train,k_test]
    print(k_train,k_test)
    df_params = pd.DataFrame({'k': k})
    # 将参数转换为DataFrame
    print(df_params)
    # 保存到CSV文件
    csv_file_path = 'submission.csv'
    df_params.to_csv(csv_file_path, index=False)